In [8]:
import pandas as pd
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.impute import SimpleImputer

# 指定数据文件路径
input_file_path = r'D:\data\data01.csv'  # 输入文件路径
output_file_path = r'D:\data\data02.csv'  # 输出文件路径

# 加载数据
try:
    diabetes_data = pd.read_csv(input_file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"文件 {input_file_path} 未找到，请检查文件路径是否正确。")

# 检查数据集是否为空
if diabetes_data.empty:
    raise ValueError("数据集为空，请检查数据加载是否正确。")

# 定义特征和目标变量
features = diabetes_data.columns[:-1]
target = diabetes_data.columns[-1]

# 遍历每个特征
for feature in features:
    print(f"正在处理特征：{feature}")
    
    # 检查是否需要处理缺失值
    if diabetes_data[feature].isnull().sum() == 0:
        print(f"特征 {feature} 没有缺失值，跳过处理。")
        continue
    
    # 拆分已知和未知值
    known = diabetes_data[diabetes_data[feature].notnull()]
    unknown = diabetes_data[diabetes_data[feature].isnull()]
    
    # 检查已知和未知数据是否为空
    if known.shape[0] == 0:
        raise ValueError(f"特征 {feature} 的已知数据为空。")
    if unknown.shape[0] == 0:
        raise ValueError(f"特征 {feature} 的未知数据为空。")
    
    # 准备训练数据
    X_train = known[features].drop(columns=[feature]).values
    y_train = known[feature].values
    X_test = unknown[features].drop(columns=[feature]).values
    
    # 创建 SimpleImputer 模型来填充目标列 y_train 的 NaN 值
    y_train_imputer = SimpleImputer(strategy='mean')
    y_train = y_train_imputer.fit_transform(y_train.reshape(-1, 1)).flatten()
    
    # 创建 HistGradientBoostingRegressor 模型
    rf_imputer = HistGradientBoostingRegressor(random_state=90)
    
    # 训练模型
    rf_imputer.fit(X_train, y_train)
    
    # 预测缺失值
    imputed_values = rf_imputer.predict(X_test)
    
    # 更新数据集中的缺失值
    diabetes_data.loc[diabetes_data[feature].isnull(), feature] = imputed_values

# 处理目标变量的缺失值
if diabetes_data[target].isnull().sum() > 0:
    print(f"正在处理目标变量：{target}")
    
    # 使用 SimpleImputer 填充目标变量的缺失值
    target_imputer = SimpleImputer(strategy='mean')
    diabetes_data[target] = target_imputer.fit_transform(diabetes_data[[target]])

# 保存处理后的数据
diabetes_data.to_csv(output_file_path, index=False)
print(f"处理后的数据已保存到 {output_file_path}")

正在处理特征：SBET/Specific surface area/m2g-1
正在处理特征：Average pore size /nm
正在处理特征：Total pore volume/cm3g-1
正在处理特征：ID
正在处理特征：IG
正在处理特征：ID/IG
正在处理特征：2θ/°
正在处理特征：d002/nm
正在处理特征：La/nm
正在处理特征：Lc/nm
正在处理特征：La/Lc
正在处理特征：first discharge/mAh g-1
正在处理特征：first charge/mAh g-1
正在处理特征：首次库伦效率/%
正在处理特征：100 次循环比容量保持率/%
正在处理特征： capacity/mAh/g
正在处理特征：Rsei/Ω
正在处理特征：Re/Ω
正在处理特征：Rf/Ω
正在处理特征：Rs/Ω
正在处理特征：Rct /Ω
正在处理特征：i0/mAcm-2
正在处理目标变量：DLi+/cm2s-1
处理后的数据已保存到 D:\data\data02.csv
